## Calculate distance between means/medoids of mutation groupings

Our goal is to find an unsupervised way of calculating distance/similarity between our mutation groupings ("none"/"one"/"both") which isn't affected by sample size, to the degree that differentially expressed gene count was (see `4_de_analysis` notebooks).

Here, we'll try the extremely simple method of:

1) taking the n-dimensional mean (centroid) or median (medoid) of each group  
2) calculating distance between the centroids and using this to define "expression similarity"

We'll try this for a few different feature selection/embedding methods, and for both gene expression and RPPA (protein expression) data.

In [1]:
from pathlib import Path
import pickle as pkl

import pandas as pd

import sys; sys.path.append('..')
import config as cfg

%load_ext autoreload
%autoreload 2

In [2]:
# whether to use expression or rppa data
# data_type = 'expression'
data_type = 'rppa'

# how to calculate centroids, 'mean' or 'median'
centroid_method = 'mean'

# number of features to subset to, by mean absolute deviation
# TODO try this in PCA/UMAP space too
subset_mad_feats = 100

### Load expression data

We'll also subset to the top features by mean absolute deviation, if that option .

In [3]:
expression_data_file = (
    '/home/jake/research/mpmp/data/tcga_expression_matrix_processed.tsv.gz'
)
expression_sample_info = (
    '/home/jake/research/mpmp/data/sample_info/tcga_expression_sample_identifiers.tsv'
)

rppa_data_file = (
    '/home/jake/research/mpmp/data/tcga_rppa_matrix_processed.tsv'
)
rppa_sample_info = (
    '/home/jake/research/mpmp/data/sample_info/tcga_rppa_sample_identifiers.tsv'
)

if data_type == 'expression':
    data_df = pd.read_csv(expression_data_file, sep='\t', index_col=0)
    sample_info_df = pd.read_csv(expression_data_file, sep='\t', index_col=0)
elif data_type == 'rppa':
    data_df = pd.read_csv(rppa_data_file, sep='\t', index_col=0)
    sample_info_df = pd.read_csv(rppa_data_file, sep='\t', index_col=0)
    
print(data_df.shape)
data_df.iloc[:5, :5]

(7790, 189)


,X1433EPSILON,X4EBP1,X4EBP1_pS65,X4EBP1_pT37T46,X53BP1
sample_id,,,,,
TCGA-OR-A5J2-01,-0.494,0.592,0.10500,-0.13500,1.89
TCGA-PA-A5YG-01,-0.535,0.183,-0.10600,-0.04010,1.03
TCGA-OR-A5JV-01,-0.109,0.418,0.02470,-0.72100,1.66
TCGA-OR-A5JT-01,-0.413,0.259,0.00597,0.56300,1.46
TCGA-OR-A5JR-01,-0.288,-0.112,0.01940,-0.00267,2.23


In [4]:
if subset_mad_feats is not None:
    mad_ranking = (
        data_df.mad(axis=0)
               .sort_values(ascending=False)
    )
    top_feats = mad_ranking[:subset_mad_feats].index.astype(str).values
    print(top_feats[:5])
    data_df = data_df.reindex(top_feats, axis='columns')
    
print(data_df.shape)
data_df.head()

['MYH11' 'VHL' 'ECADHERIN' 'ERALPHA' 'CLAUDIN7']
(7790, 100)


,MYH11,VHL,ECADHERIN,ERALPHA,CLAUDIN7,ACETYLATUBULINLYS40,RICTOR,FASN,CAVEOLIN1,GAPDH,...,MSH2,AKT,LCK,KU80,P53,SMAD1,NCADHERIN,G6PD,BAP1C4,TSC1
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5J2-01,1.950,-0.315,-1.400,-1.020,0.141,2.48,0.881,0.2280,2.160,1.550,...,-0.263,0.696,-0.0920,1.68,-0.743,0.0846,-0.24500,-0.5090,-0.427,2.25
TCGA-PA-A5YG-01,0.602,0.697,-0.223,-0.445,0.517,1.36,0.305,-0.6990,0.762,0.587,...,-0.266,0.543,-0.0364,1.44,-0.133,-0.1770,-0.18100,-0.2940,-0.531,1.11
TCGA-OR-A5JV-01,-0.679,-0.301,-1.350,-1.160,0.336,2.22,0.571,-0.2000,0.501,1.960,...,-0.402,0.437,0.2060,1.32,-0.437,0.1150,-0.25100,0.0225,-0.301,2.01
TCGA-OR-A5JT-01,-0.140,-0.481,-1.670,-1.210,0.204,2.75,0.141,-0.9400,1.450,0.503,...,-0.611,0.533,0.3390,1.10,-0.749,0.0357,0.04950,0.7790,-0.912,1.51
TCGA-OR-A5JR-01,-1.040,-0.337,-1.230,-1.090,0.265,3.04,0.331,-0.0643,0.708,1.670,...,-0.470,1.190,0.0180,1.15,-0.669,-0.1130,-0.00368,1.0000,-0.631,1.31


### Load Park et al. "hit" data

This was collated/formatted in `0_process_park.ipynb`

In [5]:
with open(cfg.distance_gain_info, 'rb') as f:
    park_gain_info = pkl.load(f)
    
park_gain_info['TP53_BRCA'].head()

,class_name,mutation_status,cnv_status,num_hits
TCGA-3C-AAAU-01,class 4,0,1,one
TCGA-3C-AALI-01,class 4,1,1,both
TCGA-3C-AALJ-01,class 4,0,1,one
TCGA-3C-AALK-01,class 4,0,1,one
TCGA-4H-AAAK-01,class 4,0,1,one


In [6]:
with open(cfg.distance_loss_info, 'rb') as f:
    park_loss_info = pkl.load(f)
    
park_loss_info['TP53_BRCA'].head()

,class_name,mutation_status,cnv_status,num_hits
TCGA-3C-AAAU-01,class 4,0,0,none
TCGA-3C-AALI-01,class 4,1,0,one
TCGA-3C-AALJ-01,class 4,0,0,none
TCGA-3C-AALK-01,class 4,0,0,none
TCGA-4H-AAAK-01,class 4,0,0,none


### Calculate distance between means/medians for given gene + cancer type

In [25]:
from scipy.spatial.distance import pdist, squareform

def get_centroids_and_distance(identifier, info_map, centroid_method='mean'):
    # get sample mutation info and class counts
    park_id_df = info_map[identifier]
    
    # get expression data for samples
    samples = park_id_df.index.intersection(data_df.index)
    park_id_df = park_id_df.reindex(samples)
    hit_class_counts = park_id_df.groupby('num_hits').count().class_name
    
    # group by number of hits, then calculate centroids
    centroids_df = (data_df
        .reindex(samples)
        .merge(park_id_df['num_hits'], left_index=True, right_index=True)
        .groupby('num_hits')
    )
    
    if centroid_method == 'mean':
        centroids_df = centroids_df.mean()
    elif centroid_method == 'median':
        centroids_df = centroids_df.median()
    else:
        raise NotImplementedError(
            'centroid method {} not implemented'.format(centroid_method)
        )
    
    # calculate distance between centroids
    dist_df = pd.DataFrame(
        squareform(pdist(centroids_df.values, metric='euclidean')),
        index=centroids_df.index.copy(),
        columns=centroids_df.index.copy()
    )
    
    return hit_class_counts, centroids_df, dist_df
    
get_centroids_and_distance('TP53_BRCA', park_loss_info, 'median')[2]

num_hits,both,none,one
num_hits,,,
both,0.000000,5.290759,2.355078
none,5.290759,0.000000,3.876024
one,2.355078,3.876024,0.000000
